10장. 딥러닝 모델 설계하기

In [ ]:
# XOR 문제를 다층 퍼셉트론으로 해경
# 하지만, 은닉층에 포함됨 가중치를 업데이트할 방법이 없음
# 이 문제를 해결하기 위한 역전파

# XOR의 경우 입력값과 출력값을 알고 있는 상태에서 가중치와 바이어스를 미리 알아본 후 적용
# 모델링이라 할 수 없음
# 데이터를 통해 스스로 가중치를 조절하는 학습 실현
# 이전 실습한 경사하강법의 경우 단일 퍼셉트론으로 입력층과 출력층만 존재할 때 가능
# 은닉층이 생기면서 두 번의 경사 하강법을 실행해야

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/mulcam_bigdata/data_모두의딥러닝

/content/drive/MyDrive/mulcam_bigdata/data_모두의딥러닝


In [ ]:
path ='./ThoraricSurgery3.csv'

In [ ]:
np.loadtxt(path, delimiter=',')

array([[ 1.  ,  2.88,  2.16, ...,  0.  , 60.  ,  0.  ],
       [ 2.  ,  3.4 ,  1.88, ...,  0.  , 51.  ,  0.  ],
       [ 2.  ,  2.76,  2.08, ...,  0.  , 59.  ,  0.  ],
       ...,
       [ 2.  ,  3.04,  2.08, ...,  0.  , 52.  ,  0.  ],
       [ 2.  ,  1.96,  1.68, ...,  0.  , 79.  ,  0.  ],
       [ 2.  ,  4.72,  3.56, ...,  0.  , 51.  ,  0.  ]])

In [ ]:
data = np.loadtxt(path, delimiter=',')
data[10]

array([ 2.  ,  2.56,  2.32,  0.  ,  0.  ,  1.  ,  0.  ,  1.  ,  0.  ,
        1.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  , 60.  ,  0.  ])

In [ ]:
# 독립변수 x, 독립변수 y

x = data[:, :-1]    # 환자의 진찰 기록을 x로 지정
y = data[:, -1]     # 수술 1년 후 환자의 생존 여부를 y로 지정

In [ ]:
# 딥러닝 모델 구조 결정

model = Sequential()
# Sequential 클래스를 사용하여 순차 모델을 생성

model.add(Dense(30, input_dim = 16, activation = 'relu')) # 은닉층
# 은닉층 / 반드시 input_dim(입력값) 포함
# activation='relu': 활성화 함수로 ReLU(Rectified Linear Unit)를 사용
# ReLU 함수는 출력값을 0 이상으로 만들어주는 비선형 활성화 함수
# >> 데이터에서 값을 16개 받아 은닉층의 노드 30개로 보낸다는 의미

model.add(Dense(1, activation = 'sigmoid'))   # 마지막 층은 결과를 출력하는 출력
# 출력층 / 이진분류 sigmoid(1/0) >> 출력 값을 0과 1 사이로 변환하여 이진분류 문제에 적
# Dense(1, ...): 출력층에 1개의 뉴런을 사용 >> 이진 분류를 수행하기 위해

# 뉴런의 개수를 결정하는 데에는 명확한 공식이 없고, 여러 요인을 고려하여 실험적으로 최적의 수를 찾아야

# model.add()라는 라인을 추가하면 새로운 층이 만들어짐
# Dense 라는 함수를 통해 구체적으로 구조가 결정
# Dense 함수의 첫 번째 인자에 몇 개의 노드를 이 층에 만들 것인지 제시
# 노드는 가중합에 해당하는 것으로 이전 층에서 전달딘 변수와 가중치, 바이어스가 하나로 모이게 됨

In [1]:
# 손실함수는 두가지 종료
# 1. 평균제곱오차 : 선형 회귀에서 사용
# 2. 교차 엔트로피 오차 : 로지스틱 회귀에서 이진분류의 예측이므로 binary_crossentropy
#                         다항 분류에서는 categorical_crossentropy

In [ ]:
# 모델 실행
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
# compile 함수를 사용하여 고급언어(사람이 사용하는)를 저급 언어(컴퓨터가 이해할 수 있는 언어)로 변
# loss='binary_crossentropy':
# 이진 분류 문제에서 흔히 사용되는 손실 함수로, 예측값과 실제값 사이의 차이를 측정

history = model.fit(x,y, epochs = 5, batch_size = 16)
# epoch : 학습 과정에서 전체 데이터 셋을 5번 순환
# batch_size : 한 번에 모델에 전달되는 데이터 샘플의 수, 16개의 샘플씩 모델에 입력되어 가중치가 업데이트
#              한 번에 몇 개씩 처리할 지 정의하는 부분으로, 16개씩 끊어서 집어 넣으라는 의미
# >> 너무 크면 학습속도가 느리고, 너무 작으면 각 실행 값의 편차가 생겨서 전체 결과값이 불안정

Epoch 1/5
30/30 [==============================] - 1s 2ms/step - loss: 0.6404 - accuracy: 0.8255
Epoch 2/5
30/30 [==============================] - 0s 2ms/step - loss: 0.4829 - accuracy: 0.8319
Epoch 3/5
30/30 [==============================] - 0s 2ms/step - loss: 0.4253 - accuracy: 0.8489
Epoch 4/5
30/30 [==============================] - 0s 2ms/step - loss: 0.4190 - accuracy: 0.8511
Epoch 5/5
30/30 [==============================] - 0s 2ms/step - loss: 0.4228 - accuracy: 0.8511
